In [1]:
import gradio as gr

from retrieval import *


# Chatbot function
def chatbot_interface(user_input):
    # Generate text response and relevant video segment
    chunks = grab_relevant_chunks(user_input)  # Generate text response
    video_segment_path = None
    text = ""
    
    for chunk in chunks:
        video_idx = chunk["video"]
        start = chunk["start"]
        end = chunk["end"]
        text = chunk["text"]
        try:
            video_segment_path = get_video_segment(start, end, video_idx)
        except:
            video_segment_path = None
    
    chat_history = [
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": generate_response(user_input, text)}
    ]
    return (chat_history, video_segment_path)

# Interface elements
with gr.Blocks(title="Video-Based Q&A Bot") as iface:
    gr.Markdown("## Ask any question about video content and get answers based on the video segments.")
    user_input = gr.Textbox(label="Ask a Question", placeholder="Type your question here...", lines=1)
    submit_btn = gr.Button("Submit Question")
    chatbot_output = gr.Chatbot(label="Chatbot", type="messages")
    video_output = gr.Video()

    # Set button click to trigger chatbot
    submit_btn.click(fn=chatbot_interface, inputs=user_input, outputs=[chatbot_output, video_output])

iface.launch()

2025-05-06 05:30:58.687647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746523858.703024  142269 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746523858.707635  142269 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746523858.719948  142269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746523858.719971  142269 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746523858.719972  142269 computation_placer.cc:177] computation placer alr

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena